<a href="https://colab.research.google.com/github/JairusTheAnalyst/JairusTheAnalyst/blob/main/Mini_Project_6_Basic_Recommendation_Systems.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from sklearn.metrics import mean_squared_error

# Simulate a dataset of user ratings
np.random.seed(42)
num_users = 10
num_products = 8

# Create a ratings matrix with some missing values (represented by NaN)
ratings = np.random.randint(1, 6, size=(num_users, num_products)).astype(float)
mask = np.random.choice([0, 1], size=ratings.shape, p=[0.3, 0.7]).astype(bool)
ratings[~mask] = np.nan

print("Original Ratings Matrix:")
print(ratings)

# Low-rank matrix approximation using SVD
def low_rank_approximation(ratings, rank):
    # Step 1: Fill NaNs with column means
    filled_ratings = ratings.copy()
    col_means = np.nanmean(filled_ratings, axis=0)
    inds = np.where(np.isnan(filled_ratings))
    filled_ratings[inds] = np.take(col_means, inds[1])

    # Step 2: Apply SVD
    U, S, VT = np.linalg.svd(filled_ratings, full_matrices=False)
    S_reduced = np.diag(S[:rank])
    U_reduced = U[:, :rank]
    VT_reduced = VT[:rank, :]

    # Step 3: Reconstruct matrix with rank-k approximation
    approx_ratings = np.dot(U_reduced, np.dot(S_reduced, VT_reduced))
    return approx_ratings

# Predict missing ratings
rank = 2
predicted_ratings = low_rank_approximation(ratings, rank)

print("\nPredicted Ratings Matrix:")
print(np.round(predicted_ratings, 2))

# Accuracy analysis: compare known values
def compute_mse(true_ratings, predicted_ratings, mask):
    true_values = true_ratings[mask]
    predicted_values = predicted_ratings[mask]
    return mean_squared_error(true_values, predicted_values)

mse = compute_mse(ratings, predicted_ratings, mask)

print("\nMean Squared Error of Predictions:", round(mse, 4))

# Recommendation: Recommend top products for a user with missing ratings
user_id = 0
user_ratings = ratings[user_id]
pred_user_ratings = predicted_ratings[user_id]

# Recommend items where original rating is missing
missing_indices = np.isnan(user_ratings)
recommended_products = np.argsort(pred_user_ratings[missing_indices])[::-1]
recommended_product_ids = np.where(missing_indices)[0][recommended_products]

print(f"\nRecommended Products for User {user_id}: {recommended_product_ids.tolist()}")

# Scalability analysis
print("\nScalability Analysis:")
print("Number of Users:", num_users)
print("Number of Products:", num_products)
print("Matrix Rank Used:", rank)


Original Ratings Matrix:
[[nan nan nan nan  5.  2. nan nan]
 [nan  5.  4.  3.  5.  2.  4.  2.]
 [ 4. nan  1.  4.  2.  5. nan  1.]
 [ 1.  3.  3.  2. nan nan nan  4.]
 [nan  1. nan  5.  3.  5.  1. nan]
 [ 4.  1. nan  2.  2.  1.  2.  5.]
 [nan nan  4. nan  4.  5.  3. nan]
 [ 4.  2. nan  2. nan  4.  5. nan]
 [ 2.  4.  2. nan nan nan nan  5.]
 [ 5. nan nan nan  1.  4. nan  4.]]

Predicted Ratings Matrix:
[[2.82 3.36 3.19 2.44 3.98 2.57 3.44 3.58]
 [2.75 4.1  3.73 2.3  4.88 2.17 4.04 4.  ]
 [3.78 1.51 2.04 3.57 1.74 4.65 2.15 2.98]
 [2.62 2.9  2.8  2.29 3.43 2.48 3.02 3.19]
 [4.16 1.66 2.25 3.92 1.92 5.1  2.37 3.28]
 [2.58 2.25 2.31 2.32 2.65 2.68 2.47 2.78]
 [3.87 2.99 3.17 3.5  3.52 4.16 3.39 3.93]
 [3.52 2.89 3.01 3.17 3.4  3.72 3.22 3.68]
 [3.15 3.07 3.06 2.79 3.63 3.14 3.29 3.59]
 [3.95 2.09 2.52 3.67 2.44 4.64 2.67 3.43]]

Mean Squared Error of Predictions: 0.9997

Recommended Products for User 0: [7, 6, 1, 2, 0, 3]

Scalability Analysis:
Number of Users: 10
Number of Products: 8
Matri